In [1]:
#!/usr/bin/python2.7

import argparse
import math
import numpy as np
import os
import pandas as pd
import random
import shutil
import sys
import gzip

#sys.path.append("/san/melissa/workspace/str-qtl/lmm/")
#from LMMSimulationsUtils2 import *

"""
Analyze heritability of gene expression due to SNPs/STRs
Notes:
** The reported SE for STRs when treating the STR as fixed effect are the SE on *beta*, not on *beta^2* **
"""

GCTA_MIN_VE = 0.000001 # Lowest VE that GCTA reports when constrained

def PROGRESS(msg, printit=True):
    if printit: # false for some messages when not in debug mode
        sys.stderr.write("%s\n"%msg.strip())

def GetMAF(x):
    """
    Get SNP MAF
    """
    x=x.convert_objects(convert_numeric=True)
    vals = x.values
#    print(len(vals))
    maf = sum(vals)*1.0/(2*len(vals))
    return min([maf, 1-maf])

def GRM(snpdata):
    """
    Return n by n GRM, scaled to have mean diagonal element 1
    """
#    snpdata.apply(pd.to_numeric, errors='coerce')
    p, n = snpdata.shape
    print(p,'  ',n)
    K = np.zeros((n, n))
    for i in range(p):
#        print(snpdata.iloc[i,:])
        gt = snpdata.iloc[i,:].apply(lambda x: float(x))
        var = np.var(gt)
        #print(var)
        if var == 0: continue
        m = np.mean(gt)
        x = np.reshape((np.array(gt)-m).transpose(), (n, 1))
        gt_m = 1/(var*p)*x.dot(x.transpose())
        K = K + gt_m
#    print(K )
    # Make sure diagonal had mean 1 (should anyway)
    diag_mean = np.mean(np.diagonal(K))
#    print(np.diagonal(K))
    K = K/diag_mean
    return K

def WriteGCTAPhenotypeFile(locus, exprfile):
    """
    Write the phenotype file 
    """
    n=len(locus)
    f = open(exprfile, "w")
    for i in range(n):
        f.write(' '.join([str(i),str(i),str(locus[i]),'\n']))
    f.close()

def WriteGCTAGRM(K, grmfile, p):
    """
    Calculate GRM and output file in GCTA format (.gz)
    Need to write:
    $grmfile.grm.gz: ind1, ind2, num nonmissing SNPs, relatedness (space-separated) (indices start at 1, rows into $grmfile.ind)
      only includes lower triangle of GRM
    $grmfile.ind: family ID, individual ID (space-separated)
    """
    n = K.shape[0]
    # Calculate GRM
    f = gzip.open("%s.grm.gz"%grmfile, "wb")
    for i in range(n):
        for j in range(i, n):
            val = K[i, j]
#it was     f.write(" ".join(map(str, [j+1, i+1, p, val]))+"\n")
            f.write(bytes(" ".join(map(str, [j+1, i+1, p, val]))+"\n" , 'UTF-8'))
    f.close()
    # Write ind file
    f = open("%s.grm.id"%grmfile, "w")
    for i in range(n):
        f.write(" ".join(map(str, [i, i]))+"\n")
    f.close()

def ParseGCTAResults(gctafile, include_str):
    """
    return cis_snp_h2, cis_snp_h2_se, cis_str_h2, cis_str_h2_se, logL
    """
    f = open(gctafile, "r")
    lines = f.readlines()
    cis_str_h2 = None
    cis_str_h2_se = None
    for line in lines:
        items = line.strip().split()
        if len(items) < 1: continue
        if include_str == "NO" or include_str == "FE" or include_str == "SAMPLES":
            if items[0] == "V(G)":
                cis_snp_h2 = items[1]
                cis_snp_h2_se = items[2]
        else:
            if items[0] == "V(G1)":
                cis_snp_h2 = items[1]
                cis_snp_h2_se = items[2]
            if items[0] == "V(G2)":
                cis_str_h2 = items[1]
                cis_str_h2_se = items[2]
        if items[0] == "logL":
            logL = items[1]
        if include_str == "FE":
            if items[0] == "Fix_eff":
                ind = lines.index(line)+2
                items = lines[ind].strip().split()
                cis_str_h2 = float(items[0])**2
                cis_str_h2_se = items[1]
        if items[0] == "Pval":
            Pval = items[1]
    if include_str == "FE":
        return cis_snp_h2, cis_snp_h2_se, cis_str_h2, cis_str_h2_se, logL, Pval
    else:
        return cis_snp_h2, cis_snp_h2_se, cis_str_h2, cis_str_h2_se, logL, 'N/A'

def GetPermutedLocusSTRs(locus_str):
    """
    Return locus_str dataframe with STR genotypes permuted
    """
    gts = list(locus_str.iloc[:,0])
    random.shuffle(gts)
    locus_str_perm = pd.DataFrame({locus_str.columns[0]: gts})
    locus_str_perm.index = locus_str.index
    return locus_str_perm

def z(vals):
    vals = list(map(float, list(vals)))
    m = np.mean(vals)
    s = math.sqrt(np.var(vals))
    return [(item-m)*1.0/s for item in vals]

def ZNorm(locus_vars):
    """
    Znormalize variants
    """
    columns = locus_vars.columns
    for c in columns:
        locus_vars[c] = z(locus_vars[c])
    return locus_vars


def WriteGCTACovarFile(locus, strcovarfile):
    """ Write GCTA covariable file using normalized genotype
    """
    f = open(strcovarfile, "w")
    n=locus.shape[0]
    for i in range(n):
        N_geno = list(locus.iloc[i,:].values)
        f.write(" ".join([str(i), str(i)]+[str(m) for m in N_geno])+"\n")
    f.close()

In [2]:
    EXPRFILE = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/Corr_Expr.csv"
    EXPRANNOTFILE = "~/projects/GTEX_eSTRs/data/Lin_Reg/Gene_Exp_Annotation.txt"
    CHROM = "chr22"
    REML_NO_CONSTRAIN=True
    if "chr" not in str(CHROM): CHROM="chr%s"%CHROM
    DISTFROMGENE = 10000
    STRGTFILE = "/storage/szfeupe/Runs/GTEx_estr/Normalized_Genotypes/STR_Norm_Gen.chr22"
    SNPGTFILE = "/storage/szfeupe/Runs/GTEx_estr/SNP_Analysis/rawnormgen1"
    OUTFILE = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/HH/Testingcenter/h2_gtca"
    TMPDIR = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/HH/Testingcenter/tmp"
    SNPMAF = 0.05
    LMM_METHOD = "GCTA"
    INCLUDE_STR = "FE"
    ESTR_RESULTS_FILE = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/PQValue.tsv"
    ESTR_GENES_ONLY = 0.1
    UNLINKED_CTRL=True

In [3]:
# Load expression and annotation
expr = pd.read_csv(EXPRFILE)

#
expr_annot = pd.read_csv(EXPRANNOTFILE)
expr_annot.index = expr_annot["probe.id"].values
expr_annot = expr_annot[expr_annot["gene.chr"] == CHROM]
#
expr_annot = expr_annot.loc[[ge for ge in expr_annot.index if ge in list(expr.columns)]]

# Load SNP genotypes
snpgt = pd.read_csv(SNPGTFILE, sep="\t", header=None)
##########TODOremove the header=None above and the next 3 lines
header="chrom     start   GTEX-QV31  GTEX-R55E       GTEX-X4XY       GTEX-X62O       GTEX-N7MS       GTEX-N7MT       GTEX-NFK9       GTEX-NL3H       GTEX-NL4W       GTEX-NPJ7 GTEX-NPJ8       GTEX-O5YT       GTEX-O5YV       GTEX-O5YW       GTEX-OHPK       GTEX-OHPM       GTEX-OHPN       GTEX-OIZF       GTEX-OIZG       GTEX-OIZH       GTEX-OIZI    GTEX-OOBK       GTEX-OXRK       GTEX-OXRL       GTEX-OXRN       GTEX-OXRO       GTEX-OXRP       GTEX-P44H       GTEX-P4PP       GTEX-P4PQ       GTEX-P4QR       GTEX-P4QS     GTEX-P4QT       GTEX-P78B       GTEX-PLZ4       GTEX-PLZ5       GTEX-PLZ6       GTEX-POMQ       GTEX-POYW       GTEX-PSDG       GTEX-PVOW       GTEX-PW2O       GTEX-PWCY    GTEX-PWN1       GTEX-PWO3       GTEX-PWOO       GTEX-Q2AG       GTEX-Q2AH       GTEX-Q2AI       GTEX-Q734       GTEX-QEG4       GTEX-QEG5       GTEX-QESD       GTEX-QLQW    GTEX-QMRM       GTEX-QV44       GTEX-QVUS       GTEX-QXCU       GTEX-R3RS       GTEX-R55C       GTEX-R55G       GTEX-REY6       GTEX-RM2N       GTEX-RN64       GTEX-RNOR     GTEX-RU1J       GTEX-RU72       GTEX-RUSQ       GTEX-RWS6       GTEX-RWSA       GTEX-S32W       GTEX-S341       GTEX-S4P3       GTEX-S4Q7       GTEX-S4UY       GTEX-S4Z8    GTEX-S7PM       GTEX-S7SE       GTEX-S95S       GTEX-SIU7       GTEX-SIU8       GTEX-SN8G       GTEX-SNOS       GTEX-SSA3       GTEX-SUCS       GTEX-T2IS       GTEX-T2YK  GTEX-T5JC       GTEX-T5JW       GTEX-T6MN       GTEX-T6MO       GTEX-T8EM       GTEX-TKQ2       GTEX-TML8       GTEX-TMMY       GTEX-TMZS       GTEX-TSE9       GTEX-U3ZG    GTEX-U3ZH       GTEX-U3ZN       GTEX-U4B1       GTEX-U8XE       GTEX-UJHI       GTEX-UJMC       GTEX-UPIC       GTEX-UPJH       GTEX-UPK5       GTEX-UTHO       GTEX-V955     GTEX-VJYA       GTEX-VUSH       GTEX-W5WG       GTEX-W5X1       GTEX-WCDI       GTEX-WFG7       GTEX-WFJO       GTEX-WFON       GTEX-WHPG       GTEX-WHSB       GTEX-WHWD     GTEX-WK11       GTEX-WL46       GTEX-WOFM       GTEX-WVLH       GTEX-WWYW       GTEX-WXYG       GTEX-WY7C       GTEX-WYJK       GTEX-WZTO       GTEX-X261       GTEX-X3Y1  GTEX-X4EO       GTEX-X4XX       GTEX-X585       GTEX-X8HC       GTEX-XBEC       GTEX-XBED       GTEX-XGQ4       GTEX-XMK1       GTEX-XOT4       GTEX-XOTO       GTEX-XPT6 GTEX-XQ3S       GTEX-XUJ4       GTEX-XUW1       GTEX-XV7Q       GTEX-XXEK       GTEX-XYKS"     
header = header.split()
snpgt.columns=header
##########END TODO

# Load STR genotypes
strgt = pd.read_csv(STRGTFILE, sep="\t")

# Restrict to STR samples
str_samples = list(set(strgt.columns[2:].values).intersection(set(snpgt.columns[2:].values)))
expr = expr.loc[str_samples,:]
snpgt = snpgt[["chrom","start"] + str_samples]
strgt = strgt[["chrom","start"] + str_samples]

# Load STR results
if ESTR_RESULTS_FILE is not None:
    estr_results = pd.read_csv(ESTR_RESULTS_FILE, sep='\t')
    
# Set up output file
outf = open(OUTFILE, "w")
if INCLUDE_STR == "NO" or INCLUDE_STR == "SAMPLES":
    outf.write("\t".join(["chrom","gene","num_snps","cis_snp_h2","cis_snp_h2_se","logL","nsamp"])+"\n")
    print(0)
elif INCLUDE_STR == "FE":
    outf.write("\t".join(["chrom","gene","str_start","num_snps","cis_snp_h2","cis_snp_h2_se","cis_str_h2", "cis_str_h2_se", "logL","pval"])+"\n")
    print(1)

# Restrict to eSTR genes
if ESTR_GENES_ONLY < 1:
    genelist = set(estr_results[estr_results["qvalue"]<=ESTR_GENES_ONLY].gene)
    genelist = [item for item in genelist if item in expr_annot.index]
    genelist = ['ENSG00000100292.12']
    expr_annot = expr_annot.loc[genelist]

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,17,18,20,23,24,25,26,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,57,59,61,63,65,66,69,70,71,72,74,75,76,77,78,79,80,81,82,83,85,86,88,89,92,94,95,96,97,98,99,100,102,105,106,110,112,113,116,117,118,119,124,125,126,128,129,130,131,134,136,138,141,142,143,144,145,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,30,47,80,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1


In [4]:
estr_results = estr_results.loc[estr_results['gene']=='ENSG00000100292.12']#.isin(genelist)]
expr = expr['ENSG00000100292.12']#[genelist]

# For each gene, pull out data and perform specified method
for i in range(expr_annot.shape[0]):
    gene = 'ENSG00000100292.12' #expr_annot.index.values[i]
    ensgene = expr_annot["gene.id"].values[i]
    print("Getting data for %s"%gene)
    genedir = os.path.join(TMPDIR,"%s/"%gene)
    if not os.path.exists(genedir):
        os.mkdir(genedir)
    start = expr_annot["gene.start"].values[i]
    end = expr_annot["gene.stop"].values[i]
# Pull out STRs
    samples_to_keep = str_samples
    best_str_start = None
    if INCLUDE_STR != "NO":
        try:
            if UNLINKED_CTRL:
                possible_starts = list(strgt[(strgt["start"] >= (start-DISTFROMGENE)) & (strgt["start"] <= (end+DISTFROMGENE))].start)
                best_str_start = random.sample(possible_starts, 1)[0]
            else:
# make sure to match on Ensembl gene (gene is ILMN if using array)
                best_str_start = estr_results[estr_results["gene"]==ensgene].sort("p.wald")["str.start"].values[0]
        except:
            print("[%s]\tERROR: couldn't find STR LMM results"%gene)
            continue
        try:
            cis_strs = strgt[(strgt["start"] >= (start-DISTFROMGENE)) & (strgt["start"] <= (end+DISTFROMGENE))]
            locus_str = cis_strs[samples_to_keep].transpose()
#            locus_str = strgt[(strgt["start"] == best_str_start)].iloc[[0],:][str_samples].transpose()
        except:
            print("[%s]\tERROR: couldn't find STR genotypes for position %s"%(gene, best_str_start))
            continue
        locus_str.index = str_samples
        locus_str.columns = ["STR_%s"%best_str_start]
        ###So far we only choose the best str as fixed effect.But we should consider all cis STRs 
        samples_to_keep = [str_samples[k] for k in range(len(str_samples)) if str(locus_str.iloc[:,0].values[k]) != "None"]
        locus_str = locus_str.loc[samples_to_keep,:]
# Make sure STRs are normalized
        try:
            locus_str = ZNorm(locus_str)
        except:
            print("[%s]\tERROR: couldn't Z normalize STR genotypes"%(gene))
            continue
    # Pull out SNPs
    cis_snps = snpgt[(snpgt["start"] >= (start-DISTFROMGENE)) & (snpgt["start"] <= (end+DISTFROMGENE))]
    locus_snp = cis_snps[samples_to_keep].transpose()
    locus_snp.index = samples_to_keep
    locus_snp = locus_snp.dropna(axis=1, how='any')
    locus_snp.columns = cis_snps["start"].apply(lambda x: "SNP_%s"%x)
    #locus_snp_maf = locus_snp.apply(lambda x: GetMAF(x), 0)
    #print(locus_snp)
#    
#    if len(locus_snp_maf) == 0:
#        continue    
#    
#    locus_snp = locus_snp.loc[:,[i for i in range(len(locus_snp_maf)) if locus_snp_maf[i]>=SNPMAF]]
    if locus_snp.shape[1] == 0:
        print("[%s]\tERROR: no common SNPs in region"%gene)
        continue
# Get expression
    y = pd.DataFrame({"expr":list(expr.loc[:,gene])})
    y.index = str_samples
    locus_y = y.loc[samples_to_keep,["expr"]]
# Z normalize
    locus_y = (locus_y - np.mean(locus_y))/math.sqrt(np.var(locus_y))
# Make SNP GRM
    locus_snp=locus_snp.apply(pd.to_numeric, errors='coerce')
    locus_snp = locus_snp.dropna(axis=1, how='any')
    K = GRM(locus_snp.transpose())
    if str(np.mean(K)) == "nan":
        print("[%s]\tERROR: nans in GRM"%gene)

# Write GRM
    if LMM_METHOD == "GCTA":
        exprfile = os.path.join(genedir, "expr.pheno")
        mgrmfile = os.path.join(genedir, "mgrm.txt")
        snpgrmfile = os.path.join(genedir, "snp.grm.txt")
        if REML_NO_CONSTRAIN:
            reml_command = "--reml-no-constrain"
        else: reml_command = "--reml"
        gcta_cmd = "/storage/resources/source/gcta64 %s --mgrm-gz %s --pheno %s --out %s/gcta "%(reml_command, mgrmfile, exprfile, genedir)
        g = open(mgrmfile, "w")
        g.write(snpgrmfile+"\n")
        WriteGCTAGRM(K, snpgrmfile, p=locus_snp.shape[1])
        if INCLUDE_STR == "FE": # --qcovar
            strcovarfile = os.path.join(genedir, "str.qcovar")
            WriteGCTACovarFile(locus_str, strcovarfile)
            gcta_cmd += " --qcovar %s --reml-est-fix"%strcovarfile
        if INCLUDE_STR == "RE":
            K_str = GRM(locus_str.transpose())
            strgrmfile = os.path.join(genedir, "str.grm.txt")
            g.write(strgrmfile+"\n")
            WriteGCTAGRM(K_str, strgrmfile, p=locus_str.shape[1])
        g.close()
        locus_y["expr"].fillna("NA", inplace=True)
        WriteGCTAPhenotypeFile(locus_y["expr"].values, exprfile)
        gcta_cmd += " > /dev/null 2>&1"
        os.system(gcta_cmd)
# Parse results
        gctafile = os.path.join(genedir, "gcta.hsq")
        if not os.path.exists(gctafile):
            print("[%s]\tERROR: GCTA could not analyze this gene"%gene)
            continue
        cis_snp_h2, cis_snp_h2_se, cis_str_h2, cis_str_h2_se, logL, pval = ParseGCTAResults(gctafile, INCLUDE_STR)
        # Output results
        if INCLUDE_STR == "NO" or INCLUDE_STR == "SAMPLES":
            word="\t".join(map(str, [CHROM, gene, locus_snp.shape[1], cis_snp_h2, cis_snp_h2_se, logL, len(samples_to_keep)]))
            print(word)
            #outf.write(word+"\n")
        else:
            #outf.write("\t".join(map(str, [CHROM, gene, best_str_start, locus_snp.shape[1], cis_snp_h2, cis_snp_h2_se,\
            #                                   cis_str_h2, cis_str_h2_se, logL, len(samples_to_keep), pval]))+"\n")  #Figure out this , len(cis_str_h2_null)       
            print('He')
        break
outf.close()       

Getting data for ENSG00000100292.12


ValueError: Length mismatch: Expected axis has 25 elements, new values have 1 elements

In [ ]:
locus_str

In [ ]:
import pandas as pd
see=pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/SNP_Analysis/SNP_Norm_Raw_Genotypes.txt', sep='\t')
#see['ID']='str_'+see[1].astype(str)
see.shape

In [ ]:
estr_results[estr_results["gene"]==ensgene].sort("p.wald")["str.start"].values[0]